In [18]:
import pandas as pd
import time
import requests
from mistralai import Mistral
from mistralai.models import SDKError  # ✅ Correction ici !

# ✅ Initialisation du client avec la clé API
API_KEY = "W028Ch2xLau3I03ivOHxSe4zUOt9BZ1d"  # Remplace par ta vraie clé API
AGENT_ID = "ag:b909df0a:20250310:untitled-agent:d426e319"  # Remplace par ton ID d'agent

client = Mistral(api_key=API_KEY)

# ✅ Charger les tweets
df = pd.read_csv(r"C:\Users\firas\Documents\ynov\B2\challenge 48h\data\cleaned_tweets.csv")

# ✅ Vérifier si la colonne "sentiment" existe déjà
if 'sentiment' not in df.columns:
    df['sentiment'] = ""

# ✅ Fonction pour analyser le sentiment avec gestion des erreurs API
def get_sentiment(tweet):
    content = "Analyse du sentiment du tweet : " + tweet
    max_retries = 5  # ✅ Nombre maximum de tentatives en cas d'erreur
    retries = 0

    while retries < max_retries:
        try:
            response = client.agents.complete(
                agent_id=AGENT_ID,
                messages=[{"role": "user", "content": content}]
            )
            return response.choices[0].message.content.strip()

        except SDKError as e:
            if "429" in str(e):  # ✅ Gestion du dépassement de quota API
                print("⚠️ Trop de requêtes envoyées ! Attente de 60 secondes avant de réessayer...")
                time.sleep(60)
            else:
                print(f"🚨 Erreur API : {e}")
                return "Erreur API"

        except requests.exceptions.Timeout:
            print("⏳ Timeout détecté ! Nouvelle tentative dans 10 secondes...")
            time.sleep(10)

        except requests.exceptions.RequestException as e:
            print(f"❌ Erreur réseau : {e}")
            return "Erreur Réseau"

        except Exception as e:
            print(f"🔴 Erreur inconnue : {e}")
            return "Erreur Inconnue"

        retries += 1

    return "Erreur après plusieurs tentatives"

# ✅ Appliquer la fonction et sauvegarder les résultats régulièrement
for idx, row in df.iterrows():
    if df.at[idx, 'sentiment'] == "":  # ✅ Ne pas réanalyser les tweets déjà traités
        print(f"🔄 Traitement du tweet {idx+1}/{len(df)}...")
        df.at[idx, 'sentiment'] = get_sentiment(row['tweet_text'])
        time.sleep(2)  # ✅ Pause réduite pour accélérer le traitement

        # ✅ Sauvegarde toutes les 10 tweets pour éviter les pertes
        if idx % 10 == 0:
            df.to_csv(r"C:\Users\firas\Documents\ynov\B2\challenge 48h\data\cleaned_tweets_with_sentiment.csv", index=False)
            print("💾 Sauvegarde intermédiaire effectuée !")

# ✅ Sauvegarde finale
df.to_csv(r"C:\Users\firas\Documents\ynov\B2\challenge 48h\data\cleaned_tweets_with_sentiment.csv", index=False)
print("\n✅ Analyse terminée ! Fichier sauvegardé avec les sentiments.")


🔄 Traitement du tweet 1/569...
🔴 Erreur inconnue : [Errno 11004] getaddrinfo failed
💾 Sauvegarde intermédiaire effectuée !
🔄 Traitement du tweet 2/569...
🔄 Traitement du tweet 3/569...
🔄 Traitement du tweet 4/569...
🔄 Traitement du tweet 5/569...
🔄 Traitement du tweet 6/569...
🔄 Traitement du tweet 7/569...
🔄 Traitement du tweet 8/569...
🔄 Traitement du tweet 9/569...
🔄 Traitement du tweet 10/569...
🔄 Traitement du tweet 11/569...
💾 Sauvegarde intermédiaire effectuée !
🔄 Traitement du tweet 12/569...
🔄 Traitement du tweet 13/569...
🔄 Traitement du tweet 14/569...
🔄 Traitement du tweet 15/569...
🔄 Traitement du tweet 16/569...
🔄 Traitement du tweet 17/569...
🔄 Traitement du tweet 18/569...
🔄 Traitement du tweet 19/569...
🔄 Traitement du tweet 20/569...
🔄 Traitement du tweet 21/569...
💾 Sauvegarde intermédiaire effectuée !
🔄 Traitement du tweet 22/569...
🔄 Traitement du tweet 23/569...
🔄 Traitement du tweet 24/569...
🔄 Traitement du tweet 25/569...
🔄 Traitement du tweet 26/569...
🔄 Trait

In [19]:
df.to_csv("sentiments.csv", index=False)

In [ ]:
df.to_csv("sentiments.csv", index=False)

In [10]:
df.to_csv("sentiments.csv", index=False)

In [13]:
import mistralai
print(dir(mistralai))

['APIEndpoint', 'Agents', 'AgentsCompletionRequest', 'AgentsCompletionRequestMessages', 'AgentsCompletionRequestMessagesTypedDict', 'AgentsCompletionRequestStop', 'AgentsCompletionRequestStopTypedDict', 'AgentsCompletionRequestToolChoice', 'AgentsCompletionRequestToolChoiceTypedDict', 'AgentsCompletionRequestTypedDict', 'AgentsCompletionStreamRequest', 'AgentsCompletionStreamRequestMessages', 'AgentsCompletionStreamRequestMessagesTypedDict', 'AgentsCompletionStreamRequestStop', 'AgentsCompletionStreamRequestStopTypedDict', 'AgentsCompletionStreamRequestToolChoice', 'AgentsCompletionStreamRequestToolChoiceTypedDict', 'AgentsCompletionStreamRequestTypedDict', 'Any', 'ArchiveFTModelOut', 'ArchiveFTModelOutObject', 'ArchiveFTModelOutTypedDict', 'Arguments', 'ArgumentsTypedDict', 'AssistantMessage', 'AssistantMessageContent', 'AssistantMessageContentTypedDict', 'AssistantMessageRole', 'AssistantMessageTypedDict', 'AsyncHttpClient', 'BaseModelCard', 'BaseModelCardTypedDict', 'BaseSDK', 'Batc

In [33]:
from mistralai import Mistral

# Initialisation du client avec la clé API
client = Mistral(api_key="QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ")

# Le contenu du tweet que tu veux analyser
content = "C'est une super journée, je suis tellement content !"

# Ajout d'un préfixe pour spécifier l'analyse de sentiment
content = "Analyse du sentiment du tweet : " + content

# Utilisation de l'agent pour obtenir la réponse
response = client.agents.complete(
    agent_id="ag:f685da10:20250310:untitled-agent:5d68c6b9",  # Remplace par ton ID d'agent
    messages=[{
        "role": "user",
        "content": content
    }]
)

# Récupérer et afficher la réponse générée par l'agent
response_text = response.choices[0].message.content.strip()
print("Sentiment analysé : ", response_text)


Sentiment analysé :  {
  "inconfort": 0,
  "sentiment": positif,
  "urgence": False
}


In [45]:
import pandas as pd

#data = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets_with_sentiment.csv")
df = pd.read_csv(r"C:\Users\Harrison\Challenge48\data\cleaned_tweets.csv")

df.columns

Index(['user_id', 'screen_name', 'name', 'date', 'tweet_text', 'hour',
       'tweet_length', 'criticite'],
      dtype='object')

In [ ]:
import pandas as pd
import requests

# Charger le DataFrame
df = pd.read_csv(r"chemin/vers/ton_fichier.csv")

# Filtrer les tweets négatifs
df_negatifs = df[df['sentiments'] == 'négatif'].copy()

# Configuration de l'API Mistral
API_KEY = "QU9joTThl5C9C8Waz8dgk2Mu9mFT7TaZ"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "User-Agent": "ag:f685da10:20250310:untitled-agent:5d68c6b9"
}

# Fonction pour interroger le modèle Mistral
def get_intent(tweet):
    prompt = f"""
    Analyse ce tweet et identifie la cause principale du sentiment négatif. 
    Précise si la négativité est due à un retard de livraison, un produit défectueux, 
    un mauvais service client ou une autre raison. Si tu n'es pas sûr, indique "autre".
    Réponds uniquement par la raison identifiée.

    Tweet : "{tweet}"
    """

    response = requests.post("https://console.mistral.ai/home ", json={"prompt": prompt}, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get("result", "autre")
    else:
        return "erreur_api"

# Appliquer la fonction sur les tweets négatifs
df_negatifs['intent'] = df_negatifs['tweets'].apply(get_intent)

# Fusionner avec le DataFrame original
df = df.merge(df_negatifs[['tweets', 'intent']], on='tweets', how='left')

# Sauvegarder le résultat
#df.to_csv("chemin/vers/fichier_sortie.csv", index=False)

print("Traitement terminé. Résultats enregistrés !")


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}